## Jupyter Notebook to fetch commitment data from an API of certain contracts

In [3]:
import requests
import pandas as pd
import json

### Makes the request for commitments based on the contract IDs

In [4]:
BASE_URL = "https://contratos.comprasnet.gov.br/api"

def get_contratos(contract_id):
    endpoint = f"/contrato/{contract_id}/empenhos"
    url = f"{BASE_URL}{endpoint}"
    response = requests.get(url)
    
    # Verificar o status da resposta
    if response.status_code != 200:
        raise Exception(f"Erro na requisição: {response.status_code}")
    
    # Verificar o conteúdo da resposta
    if not response.text:
        raise Exception("Resposta vazia")
    
    try:
        data = response.json()
    except ValueError as e:
        raise Exception(f"Erro ao decodificar JSON: {e}")
    
    return data

# Ler o arquivo contratos.csv e extrair os IDs dos contratos
contratos_df = pd.read_csv('contratos.csv')
contract_ids = contratos_df['id'].tolist()

# Inicializar uma lista para armazenar todos os empenhos
all_empenhos = []

# Fazer requisições para cada ID de contrato
for contract_id in contract_ids:
    try:
        empenhos = get_contratos(contract_id)
        all_empenhos.extend(empenhos)
    except Exception as e:
        print(f"Erro ao processar contrato {contract_id}: {e}")

# Salvar a lista de empenhos em um arquivo JSON
with open('empenhos.json', 'w') as f:
    json.dump(all_empenhos, f, indent=4)

### Transforms and nests the json data and saves it to a csv file

In [7]:
# Função para achatar dicionários aninhados
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Ler o arquivo JSON
with open('empenhos.json', 'r') as f:
    data = json.load(f)

# Achatar cada dicionário na lista
flattened_data = [flatten_dict(item) for item in data]

# Converter a lista de dicionários achatados em um DataFrame
df = pd.DataFrame(flattened_data)

# Salvar o DataFrame em um arquivo CSV
df.to_csv('empenhos.csv', index=False)